## Load data and check data structure

In this example we will use data available here: https://lambrechtslab.sites.vib.be/en/single-cell

Set working directory and load rds object

In [ ]:
setwd("/Your/Directory")
data <- readRDS("1867-counts_cells_cohort2.rds")

Load required package

In [ ]:
library("DropletUtils")

Looking at data structure we can see that this is a dgCMatrix with gene symbols as rownames and cell barcodes as colnames

In [ ]:
str(data)

Let's look at the firs cell barcode as an example (BIOKEY_13_Pre_AAACCTGCAACAACCT-1)

In [ ]:
colnames(data)[1]

In this particular case, cell barcodes consist of a prefix (BIOKEY_13_Pre_) and a sequence of bases (AAACCTGCAACAACCT-1)
The prefixes are sample names, so we'll use them to demultiplex the data

### Demultiplex data based on barcodes prefixes and export as 10X files

Use a regular expression to extract prefixes

In [ ]:
data.pfx <- gsub("(.+)_[A-Z]+-1$", "\\1", colnames(data), perl=TRUE)

Get unique sample names

In [ ]:
data.samples <- unique(data.pfx)

Check  sample names (pay attention if are using this script to process a different dataset as the regular expression may need to be modified depending on the specific colnames)

In [ ]:
head(data.samples)
tail(data.samples)

Export as 10X files that can be directly uploaded to Cellenics

Define a function that creates a subdirectory named "demultiplexed" inside the current working directory, and save 10X data for each sample in different subfolders.
If a folder named "demultiplexed" already exists, it will stop and return an error to avoid overwriting files.

In [ ]:
demultiplex_convert_to_10x <- function(obj, samples) {
          if(!dir.exists(file.path(getwd(), "demultiplexed"))) {
          dir.create(file.path(getwd(), "demultiplexed"))
        } else {
          print("WARNING! A demultiplexed directory already exists")
          return()
        }
        for (i in 1:length(samples)) {
        print(paste0("Converting sample ", samples[i]))
        DropletUtils::write10xCounts(path = paste0(getwd(),"/demultiplexed/",samples[i]), x = obj[,grep(paste0("^",samples[i],".*"),colnames(obj))], type = "sparse", version="3")
        }
}

Run the function

In [ ]:
demultiplex_convert_to_10x(obj = data, samples = data.samples)